In [198]:
import simpy
from __future__ import division

## Simpy tutorial

### Generator statements

In [199]:
def range_(n):
    i = 0
    while i < n:
        yield i
        i += 1

In [200]:
for value in range_(10):
    print(value)

0
1
2
3
4
5
6
7
8
9


In [201]:
x = range_(10)
print(x)

<generator object range_ at 0x0000017D392BF0A0>


In [202]:
print(next(x))

0


### Simpy basics

In [203]:
def car(env):
    while True:
        print('Start parking at %d' % env.now)
        parking_duration = 5
        yield env.timeout(parking_duration)
        print('Start driving at %d' % env.now)
        trip_duration = 2
        yield env.timeout(trip_duration)

In [204]:
env = simpy.Environment()
env.process(car(env))
env.run(until=15)

Start parking at 0
Start driving at 5
Start parking at 7
Start driving at 12
Start parking at 14


### In classes

In [205]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
            yield self.env.timeout(duration)

In [206]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [207]:
env = simpy.Environment()
car = Car(env)
env.process(driver(env,car))
env.run(until=15)

Start parking and charging at 0
Was interrupted. Hope, the battery is full enough ...
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


### Resources/Servers

In [208]:
def car(env, name, bcs, driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(driving_time)

    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req
        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [209]:
env = simpy.Environment()
bcs = simpy.Resource(env, capacity=2)
for i in range(4):
    env.process(car(env, 'Car %d' % i, bcs, i*2, 5))
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12


### Generator classes

In [210]:
servicetimes = []
import numpy as np

class Server(object):
    """Server with constant service times""" 
    
    def __init__(self, env):
        self.env = env
        self.server = simpy.Resource(env, capacity=2) #aantal servers
    
    def server_time(self): 
        print(len(servicetimes))
        tempvalue = np.amin(servicetimes)
        servicetimes.remove(tempvalue)
        return tempvalue
    
    def serve(self):
        serve_time = self.server_time()
        yield self.env.timeout(serve_time)
    
    _server_time = server_time
    _variabele = "Hoi"
    
class Server2(Server):
    
    def server_time(self):
        return 2

    

In [211]:
#env = simpy.Environment()
#sv1  = Server(env)
#sv2 = Server2(sv1)
#
#print(sv1.server_time())
#print(sv2.server_time())
#print(sv2._server_time())
#print(sv2._variabele)
#


In [213]:
class Customer(object):
    """
    For each customer we record the arrivaltime, 
    the response time and the departure time.

    """
    def __init__(self, env, sv):
        self.env = env
        self.sv = sv
        self.time_in_line = 0
        
        servicetimes.append(np.random.exponential(0.90,1)) # mu service time        
        
    def enqueue(self):
        #print("Arrived at %f" % (self.env.now))
        with self.sv.server.request() as request:
            self.time_in_line = self.env.now
            yield request
            self.time_in_line = self.env.now - self.time_in_line
            #print("Has waited at %f" % (self.time_in_line))
            datastorage.append(self.time_in_line)
            #print("Started serving at %f" % (self.env.now))
            yield self.env.process(self.sv.serve())
            #print("Finished serving at %f" % (self.env.now))
            

In [214]:
class CustomerGenerator(object):
    """Creates new customers"""
    def __init__(self,env,sv):
        self.env = env
        self.sv = sv
        self.max_customers = 239000
        
    
    def arrival_time(self):
        randomNumber = np.random.exponential(1/2,1) #1/lambda = interarravial time
        return randomNumber
    
    def run(self):
        customers = 0
        while customers < self.max_customers:
            arrival_time = self.arrival_time()
            yield self.env.timeout(arrival_time)
            
            customers += 1
            new_customer = Customer(self.env, self.sv)
            self.env.process(new_customer.enqueue())

In [215]:
datastorage = []
servicetimes = []

class QTExperiment(object):
    """ Easy interface to generate a queue """
    
    def __init__(self):
        self.env = simpy.Environment()
        self.server = Server(self.env)
        self.generator = CustomerGenerator(self.env, self.server)
        
    def run(self):
        self.env.process(self.generator.run())
        self.env.run()

In [216]:
datastorage = []
servicetimes = []

exp = QTExperiment().run()
#print(datastorage[100])
meandata = []
sigmadata = []
for i in range(200):
    meandata.append(np.mean(datastorage[i*1200 + 100:i*1200 + 1100]))
    sigmadata.append(np.std(datastorage[i*1200 + 100:i*1200 + 1100]))

print("mm", np.mean(meandata))
print("sm", np.std(meandata))
print("ms", np.mean(sigmadata))
print("ss", np.std(sigmadata))

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
3
2
2
4
4
3
2
3
2
1
1
1
4
5
4
5
5
5
4
3
4
3
3
2
1
3
3
2
1
1
1
1
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
3
2
3
5
5
5
5
5
4
3
6
5
4
3
3
3
2
2
5
4
3
2
2
1
1
3
2
2
2
2
2
1
1
1
1
1
2
1
1
3
2
1
1
1
1
1
1
3
4
3
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
3
2
3
2
2
3
2
1
2
2
1
1
2
2
2
4
5
5
4
3
3
2
1
2
1
1
1
1
1
1
1
1
6
5
6
5
4
3
3
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
3
3
4
4
3
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
3
2
1
1
2
1
3
2
3
2
3
3
3
3
2
3
3
2
2
1
1
3
3
3
2
4
4
3
3
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
5
4
3
2
7
6
5
4
7
6
5
4
3
4
3
4
3
2
1
9
8
9
8
9
8
7
6
5
5
5
4
3
6
5
4
4
5
4
4
4
4
3
2
5
6
5
4
3
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
3
2
1
5
4
3
2
5
4
3
2
7
6
7
7
6
5
5
5


1
1
1
1
1
4
3
5
4
3
2
3
2
1
1
1
1
1
1
5
4
3
2
4
3
2
1
1
1
1
4
3
2
2
3
2
1
1
5
4
3
2
1
2
2
2
1
1
1
1
1
1
5
4
3
2
7
6
5
4
3
4
4
3
2
1
1
1
1
2
1
7
6
5
6
5
5
4
3
2
3
2
2
1
2
4
4
3
4
5
6
5
5
4
4
3
3
2
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
2
2
2
1
3
4
3
2
1
2
1
1
1
1
1
1
2
1
1
1
1
1
3
8
9
9
8
7
7
6
6
7
6
5
4
4
4
3
5
4
5
5
4
5
4
5
6
5
4
5
4
5
4
3
3
4
3
3
2
1
2
1
2
1
1
1
7
6
5
5
4
3
2
2
3
3
2
1
3
3
3
2
1
2
1
4
3
2
1
1
3
2
1
3
2
1
1
2
2
1
3
3
2
2
2
1
1
1
1
1
2
3
3
4
3
2
2
2
2
3
2
1
1
1
3
2
1
1
1
3
2
3
2
3
2
2
1
3
3
2
1
3
2
1
2
1
1
2
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
4
3
3
2
2
1
6
6
5
4
3
5
8
7
6
5
4
4
3
3
2
2
1
1
1
2
4
5
6
5
4
4
3
5
4
4
4
3
2
2
3
2
1
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
9
8
7
7
6
6
5
4
4
4
3
3
5
5
4
3
2
2
1
4
3
2
1
1
1
5
4
4
3
2
1
2
1
1
1
1
3
2
4
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
2
1
1
1
4
4
3
4
3
9
8
9
8
8
9
8
7
8
7
6
5
6
5
4
4
5
4
3
2
2
2
3
2
3
5
5
4
3
2
2
3
3
3
2
2
4
4
3
2
2
2
3
4
3
2
2
2
2
2
1
7
6
6
5
4
3
4
3
2
2
1
1
1
1
2
1
1
1
1
1


1
1
1
1
1
1
1
2
1
1
1
1
1
5
4
3
2
1
2
2
2
1
2
2
1
3
2
2
1
5
4
3
3
3
2
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
2
1
1
6
5
4
5
4
5
4
3
3
2
1
2
1
1
1
1
4
5
4
5
6
5
6
5
4
4
3
2
7
6
6
6
7
7
6
5
4
3
5
5
6
8
7
7
6
6
7
6
5
6
6
5
5
4
3
4
3
2
5
4
5
4
5
4
3
3
3
3
2
1
3
4
4
3
2
5
4
3
2
5
4
5
9
8
7
9
8
7
7
6
6
5
4
7
6
5
4
11
10
9
9
8
7
6
5
5
5
4
5
5
5
4
3
5
4
4
3
4
3
4
5
5
4
4
7
6
5
4
5
4
3
2
7
6
5
4
3
2
2
2
4
3
2
2
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
4
3
2
2
2
4
3
4
4
3
10
9
8
7
8
7
6
6
5
7
6
5
4
8
7
6
5
6
6
5
6
6
5
6
10
11
10
9
8
7
6
7
6
9
8
7
7
6
5
5
6
5
6
5
4
5
5
4
6
5
5
5
5
5
5
5
4
3
6
6
6
7
6
5
5
4
3
5
4
3
6
5
4
6
6
6
5
5
5
6
5
4
3
3
2
1
10
9
8
7
6
5
4
4
3
2
5
4
4
3
2
1
2
3
2
1
3
2
3
3
2
4
3
2
2
1
1
3
3
2
1
3
2
3
3
2
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
5
4
5
5
4
3
2
3
2
2
1
1
3
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
5
4
5
4
5
6
8
7
6
5
10
9
8
8
10
9
8
7
7
7
6
5
6
5
4
5
4
3
3
2
2
1
4
3
3
2
2
1
3
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
3
2
2
2
3
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1

3
2
2
2
6
5
5
4
3
3
2
4
3
4
4
3
3
3
2
4
5
4
6
5
6
5
6
5
4
3
6
5
4
3
4
6
5
4
4
5
5
4
4
3
4
4
3
2
4
3
2
4
3
4
3
5
4
3
5
5
4
7
6
5
4
3
4
4
3
3
4
3
2
1
4
3
2
1
5
4
3
2
2
2
1
5
4
4
3
3
3
2
2
3
2
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
2
2
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
1
3
2
1
1
1
1
2
8
7
6
5
5
4
3
3
2
2
2
1
3
2
5
4
3
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
4
4
4
3
4
3
5
5
4
3
2
1
1
4
4
3
2
2
1
1
3
3
3
5
5
4
6
5
4
7
7
6
5
4
3
3
3
3
3
4
3
2
4
5
4
3
4
3
6
5
4
3
3
4
4
4
3
6
5
5
4
5
4
4
3
3
2
2
3
2
1
1
1
3
2
4
3
2
4
3
2
2
1
1
3
2
2
2
4
4
3
3
2
1
2
2
1
1
1
1
2
1
1
1
1
4
3
2
1
1
1
3
2
2
2
1
5
4
3
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
3
2
1
7
6
5
5
4
3
3
2
1
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
2
1
1
1
1
1
1
1
1
1
3
2
1
3
2
1
1
1
2
2
1
2
2
1
1
3
2
2
1
2
1
4
3
3
3
2
1
1
1
1
5
4
3
4
3
2
3
2
1
1
4
3
2
2
1
1
1
1
3
3
2
3
2
1
2
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
3
3
2
1
1
1
1
1
1
2
1
1
1
1
1
1
2
3
2


3
2
1
1
1
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
1
1
3
2
1
1
1
1
1
3
3
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
1
2
4
5
5
7
7
7
8
7
7
8
7
7
6
6
5
4
4
5
4
3
3
2
7
6
5
4
4
3
2
3
2
3
2
3
3
2
2
1
4
3
3
4
4
3
3
2
4
4
3
2
5
4
4
3
2
4
3
3
3
2
1
5
5
4
4
3
3
3
2
2
1
3
2
2
1
5
4
4
4
3
2
4
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
2
2
1
1
1
1
3
2
1
1
1
1
2
2
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
2
3
2
1
1
4
3
2
3
3
4
4
3
3
2
2
4
3
4
3
2
1
5
6
6
7
8
7
6
5
4
5
5
6
5
5
7
6
5
7
6
5
4
4
6
5
4
3
4
6
6
7
6
5
5
4
3
3
2
4
4
4
3
2
2
2
3
2
1
6
5
4
3
6
5
5
5
9
8
8
10
9
8
7
7
7
6
8
7
7
6
5
4
4
3
6
6
5
4
4
6
5
4
4
3
3
2
2
1
3
2
3
2
1
2
2
1
1
6
6
5
4
4
3
5
4
5
5
4
3
7
7
6
5
4
3
3
2
2
4
3
3
4
3
3
2
5
4
5
4
3
4
3
2
1
4
4
4
5
4
3
2
3
2
1
3
2
1
1
1
4
3
2
1
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
1
1
3
6
6
5
4
4
5
4
4
5
7
6
6
5
4
3
3
2
3
4
7
6
7
6
6
5
4
7
6
6
5
6
5
4
3
3
4
5
4
4
6
5
4
3
5
4
5
4
3
4
3
2
1
3
2
2
2
1
3
2
6
5
4
3
3
5
4
3
4
4
3
4
4
3
2
3
3
2
1
1
1
1
1
1
1

...

1
1
1
1
3
2
1
1
1
1
1
1
1
1
1
1
1
2
1
2
4
4
3
3
5
4
4
3
2
6
5
4
5
4
3
2
2
1
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
3
3
2
3
2
2
1
1
1
1
1
1
1
1
1
1
1
1
4
3
2
1
1
1
1
2
1
1
1
1
2
2
3
2
2
1
1
1
1
1
1
3
4
4
3
2
1
1
1
1
1
1
1
1
1
1
4
3
2
2
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
4
4
6
6
5
4
4
3
2
2
1
3
4
3
2
3
3
2
1
1
1
1
1
1
1
1
1
1
2
4
5
4
5
6
5
4
4
5
7
6
5
5
5
4
3
4
4
4
3
4
3
3
3
3
3
2
1
7
6
6
6
6
5
6
5
4
3
2
2
1
1
2
5
4
3
3
4
3
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
6
5
6
5
5
4
6
5
4
5
4
4
3
3
5
4
3
2
2
5
4
3
3
5
4
3
6
6
5
5
4
3
6
5
4
5
5
4
3
5
4
3
2
3
2
3
4
3
2
2
2
1
1
1
1
1
3
5
5
4
6
5
4
3
3
5
4
5
4
3
3
3
2
2
2
2
3
2
1
1
1
6
7
6
6
5
4
5
5
6
5
7
8
7
9
8
7
8
10
9
8
8
7
7
6
11
10
9
9
8
8
9
10
11
11
10
9
11
12
11
13
12
11
11
10
9
13
12
14
13
12
11
10
10
10
9
9
9
9
8
7
11
11
10
11
10
9
9
8
7
7
7
11
10
9
8
7
6
6
9
9
8
7
8
7
6
7
8
8
7
6
6
5
4
10
9
9
9
8
7
8
8
7
6
8
8
7
7
6
9
9
8
7
8
7
6
7
8
7
7
7
6
5
6
5
5
4
5
4
8
7
6
5
4
7
6
6
5
4
5
5
4
4
4
4
3
4
6
7
6
5
8
7


7
6
6
9
8
8
7
6
9
8
7
6
6
5
4
3
4
3
4
3
6
5
4
4
3
6
6
5
6
5
4
4
3
2
4
3
2
4
3
3
3
2
2
4
3
2
2
1
3
2
1
4
3
2
2
1
8
7
6
5
5
4
4
3
4
3
2
4
3
3
2
1
1
2
6
5
5
4
3
2
1
4
3
2
2
1
1
1
2
3
4
3
2
1
4
4
4
5
5
4
7
6
5
7
10
9
8
9
8
8
7
6
7
6
5
7
7
7
10
9
9
8
10
9
8
7
6
8
8
8
8
9
8
7
6
5
7
7
6
5
7
6
5
5
4
4
3
2
7
6
5
4
3
3
4
4
3
2
2
1
1
2
5
4
5
5
4
3
5
6
6
6
6
6
5
6
6
5
4
4
3
2
2
2
2
2
2
1
1
1
5
4
3
2
3
3
2
1
4
3
5
4
3
3
2
1
5
4
3
3
2
2
5
4
3
2
5
4
5
4
4
3
2
2
2
1
1
1
1
4
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
2
3
2
1
1
1
1
2
3
3
4
6
5
5
5
6
5
5
4
3
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
3
2
2
1
1
1
1
1
3
3
2
3
3
2
1
5
4
3
4
3
2
1
3
3
2
3
4
3
4
4
3
2
5
4
4
3
3
2
1
2
1
1
1
1
3
3
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
1
1
1
1
1
6
5
4
3
2
1
1
2
2
1
1
1
1
1
1
3
2
1
1
1
1
2
1
1
1
1
2
1
3
3
3
2
2
3
2
1
1
3
2
2
1
1
1
1
1
1
1
1
3
4
4
4
3
2
1
1
1
3
6
5
4
4
3
2
1
2
1
1
4
3
3
2
1
1
2
1
1
1
1
1
1
1
2
1
3
2
2
1
1
1
1
1
1
1
2
1
1
4
3
3
3
4
3
2
4
3
2
5
4
3
3
2
3
3
3
4
3
2
3
3
2
4
3
2
2
1
1
1
1
1
1
1
1
3
2
4
3
4
3
3
2
3
2

1
1
1
2
3
3
2
4
4
5
4
3
3
3
3
5
5
4
3
2
5
4
3
2
1
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
3
7
6
5
6
7
6
5
4
5
4
3
3
2
3
2
2
3
2
2
3
3
2
1
1
3
3
3
2
1
2
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
4
3
2
1
1
2
2
2
1
3
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
2
1
1
1
1
1
2
3
3
2
2
2
3
2
3
2
3
2
3
2
2
3
2
3
2
3
2
1
1
1
1
1
1
6
7
7
7
6
5
4
3
5
4
3
2
1
2
1
1
1
1
4
3
3
3
2
1
9
8
7
6
6
5
6
5
6
5
5
5
4
6
5
6
5
4
3
5
4
3
3
3
2
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
3
2
3
2
1
2
2
1
1
1
1
2
1
2
1
1
3
2
1
1
1
1
2
1
4
4
3
3
3
4
4
3
3
2
3
3
4
4
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
3
3
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
3
2
1
1
2
1
1
1
1
1
1
1
1
1
1
1
2
1
1
5
4
4
3
2
2
1
1
1
4
3
4
4
3
2
1
1
2
1
1
1
1
2
2
1
2
1
5
4
3
2
1
1
4
4
5
5
5
4
3
2
3
4
3
3
2
2
1
1
2
2
4
3
3
3
3
4
3
3
3
6
5
4
4
3
2
1
2
2
1
4
3
2
1
1
4
4
4
3
2
1
1
3
2
4
3
3
2
3
2
5
4
3
3
4
4
3
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


2
2
2
1
1
1
1
1
2
1
2
1
1
1
1
1
1
1
1
1
1
2
2
1
1
1
1
1
1
1
1
1
2
1
3
2
1
4
4
3
2
2
6
7
6
5
4
5
5
6
5
4
7
6
6
6
5
4
3
2
5
4
5
4
6
5
4
3
4
3
2
7
6
6
6
5
6
5
5
4
3
6
8
7
6
6
5
5
4
3
3
3
3
4
3
2
1
4
3
3
2
1
1
1
1
1
1
3
4
3
2
1
7
6
6
6
6
7
6
5
5
4
3
2
3
5
4
4
3
4
3
2
4
3
2
5
4
4
3
6
5
5
4
3
6
5
4
4
3
5
5
4
6
5
5
4
3
2
2
3
2
6
5
5
5
4
3
4
3
3
3
2
1
6
5
4
3
3
4
4
3
2
3
2
2
1
2
3
4
4
3
6
5
5
5
4
6
5
4
6
5
6
5
4
5
5
4
3
4
5
4
3
5
4
3
2
4
3
2
1
2
2
2
2
1
1
2
1
5
4
3
3
2
2
1
2
2
4
3
3
2
1
1
1
1
1
2
1
1
1
1
1
1
2
1
1
1
2
1
2
2
1
1
1
1
1
1
1
1
1
1
3
5
4
4
4
4
4
4
3
5
4
3
7
6
5
4
3
2
3
2
3
3
2
1
1
3
3
2
2
1
1
4
4
4
3
2
2
3
2
6
6
5
8
9
8
7
6
5
4
4
5
4
3
2
5
5
4
5
6
5
4
3
3
6
5
4
4
4
3
4
3
3
4
3
5
4
4
3
3
2
6
5
4
4
4
3
3
2
5
4
4
3
3
4
4
3
3
5
4
3
2
2
4
3
2
4
3
3
3
4
3
2
3
3
4
3
2
2
3
2
1
1
1
2
1
1
1
1
1
1
2
2
1
3
2
1
3
2
1
1
1
1
1
1
1
1
1
1
3
3
2
6
5
4
3
2
1
3
2
3
2
3
3
2
1
5
4
3
2
3
2
1
3
2
1
1
1
2
1
1
1
1
1
1
1
1
2
3
2
3
2
1
1
2
2
4
3
2
4
3
3
2
3
2
1
7
6
5
4
8
7
6
5
4
5
7
6
5
7
6
5
5
6
7
7
6
6
5
5


6
5
4
5
4
4
3
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
1
1
1
5
4
3
3
2
1
5
4
4
3
3
2
1
2
2
1
3
4
3
2
2
1
1
1
1
1
3
3
4
3
7
7
7
6
6
5
4
5
5
4
3
2
2
1
1
1
1
1
1
1
1
1
3
2
2
1
1
1
1
4
3
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
6
5
7
6
5
4
3
3
2
1
5
4
4
4
3
2
2
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
4
3
3
3
3
3
2
4
3
3
2
1
2
1
6
5
5
4
4
3
2
4
3
2
1
1
1
2
1
1
1
1
1
1
2
2
3
3
2
3
2
2
1
1
1
1
1
2
2
1
1
2
1
1
1
1
2
2
2
1
1
1
1
2
6
5
5
4
7
6
5
4
5
4
3
3
3
2
1
2
3
2
1
2
4
3
2
1
1
2
1
1
1
2
1
1
1
1
1
1
1
1
1
2
3
3
2
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
3
2
1
1
1
1
1
1
1
1
3
3
2
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
2
1
1
1
2
2
2
2
1
1
4
4
4
6
5
4
3
5
6
5
5
4
4
3
4
4
3
2
6
5
4
3
2
2
4
5
4
4
5
6
5
7
6
5
4
3
4
3
4
4
3
4
3
4
3
5
4
5
5
5
4
4
3
2
5
4
6
6
5
7
6
5
9
10
9
8
9
8
7
7
7
7
6
6
5
6
7
6
5
5
9
8
8
8
7
6
6
5
4
5
4
3
10
9
8
7
6
5
5
8
7
6
6
6
5
5
6
5
4
3
6
7
8
7
6
6
5
4
4
3
7
6
5
5
4
6
6
6
5
5
4
3
8
7
6
5
4
3
2
3
2
3
3
3
3
2
2
3
3
3
2
1
4


In [97]:
np.random.exponential(1,1)

array([0.63314414])